# 🚀 Day 2: Thread Indexing Mastery

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sdodlapati3/cuda-lab/blob/main/learning-path/week-01/day-2-thread-indexing.ipynb)

## Learning Philosophy

> **CUDA C++ First, Python/Numba as Optional Backup**

This notebook shows:
1. **CUDA C++ code** - The PRIMARY implementation you should learn
2. **Python/Numba code** - OPTIONAL for quick interactive testing in Colab

> **Note:** If running on Google Colab, go to `Runtime → Change runtime type → T4 GPU` before starting!

---

In [ ]:
# Verify CUDA is available
!nvcc --version
!nvidia-smi --query-gpu=name,memory.total,compute_cap --format=csv

---

# Day 2: Thread Indexing Mastery

**The most critical skill** in CUDA programming is understanding how to map threads to data.

Today you'll learn:
- The CUDA thread hierarchy: Grid → Blocks → Threads
- 1D indexing formula and boundary checking
- Grid-stride loops for any array size
- 2D indexing for matrices and images

---

## 1. The Thread Hierarchy

CUDA organizes threads into a **3-level hierarchy**:

```
┌─────────────────────────────────────────────────────────────────┐
│                           GRID                                  │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐             │
│  │   Block 0   │  │   Block 1   │  │   Block 2   │  ...        │
│  │ ┌─┬─┬─┬─┐   │  │ ┌─┬─┬─┬─┐   │  │ ┌─┬─┬─┬─┐   │             │
│  │ │T│T│T│T│   │  │ │T│T│T│T│   │  │ │T│T│T│T│   │             │
│  │ │0│1│2│3│   │  │ │0│1│2│3│   │  │ │0│1│2│3│   │             │
│  │ └─┴─┴─┴─┘   │  │ └─┴─┴─┴─┘   │  │ └─┴─┴─┴─┘   │             │
│  └─────────────┘  └─────────────┘  └─────────────┘             │
└─────────────────────────────────────────────────────────────────┘

Built-in variables:
- gridDim.x     = Number of blocks in grid (3 in this example)
- blockDim.x    = Number of threads per block (4 in this example)
- blockIdx.x    = Current block index (0, 1, or 2)
- threadIdx.x   = Thread index within block (0, 1, 2, or 3)
```

## 2. 1D Indexing Formula

To find the **global index** of a thread:

```cpp
int idx = blockIdx.x * blockDim.x + threadIdx.x;
```

**Example:** Block 2, Thread 3 with 4 threads per block:
```
idx = 2 * 4 + 3 = 11
```

Let's visualize this:

In [ ]:
%%writefile thread_indexing_1d.cu
#include <stdio.h>
#include <cuda_runtime.h>

// Kernel that prints thread indexing information
__global__ void printIndices() {
    // Calculate global thread index
    int globalIdx = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Each thread prints its indices
    printf("Block %d, Thread %d -> Global Index %d\n",
           blockIdx.x, threadIdx.x, globalIdx);
}

int main() {
    printf("=== 1D Thread Indexing Demo ===\n\n");
    
    // Launch with 3 blocks, 4 threads per block
    int blocksPerGrid = 3;
    int threadsPerBlock = 4;
    
    printf("Grid: %d blocks, %d threads/block = %d total threads\n\n",
           blocksPerGrid, threadsPerBlock, blocksPerGrid * threadsPerBlock);
    
    printIndices<<<blocksPerGrid, threadsPerBlock>>>();
    cudaDeviceSynchronize();
    
    printf("\nFormula: idx = blockIdx.x * blockDim.x + threadIdx.x\n");
    
    return 0;
}

In [ ]:
!nvcc -o thread_indexing_1d thread_indexing_1d.cu && ./thread_indexing_1d

## 3. Boundary Checking: Critical for Safety!

**Problem:** We often launch more threads than we have data elements.

```
Array size: 10 elements
Threads launched: 3 blocks × 4 threads = 12 threads

Threads 0-9: Process elements (valid)
Threads 10-11: Must do NOTHING! (out of bounds)
```

**Always include boundary checks:**

```cpp
if (idx < n) {
    // Safe to access array[idx]
}
```

In [ ]:
%%writefile boundary_check.cu
#include <stdio.h>
#include <cuda_runtime.h>

// Kernel with proper boundary checking
__global__ void safeSquare(float* input, float* output, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    // CRITICAL: Check bounds before accessing memory
    if (idx < n) {
        output[idx] = input[idx] * input[idx];
    }
    // Threads with idx >= n do nothing (safe!)
}

int main() {
    const int N = 10;
    const int THREADS_PER_BLOCK = 4;
    
    // Calculate grid size (rounds up!)
    int blocksPerGrid = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;
    int totalThreads = blocksPerGrid * THREADS_PER_BLOCK;
    
    printf("=== Boundary Checking Demo ===\n\n");
    printf("Array size:      %d elements\n", N);
    printf("Threads/block:   %d\n", THREADS_PER_BLOCK);
    printf("Blocks:          %d\n", blocksPerGrid);
    printf("Total threads:   %d\n", totalThreads);
    printf("Extra threads:   %d (will be idle)\n\n", totalThreads - N);
    
    // Allocate and initialize host memory
    float h_input[N], h_output[N];
    for (int i = 0; i < N; i++) {
        h_input[i] = i + 1;  // 1, 2, 3, ..., 10
    }
    
    // Allocate device memory
    float *d_input, *d_output;
    cudaMalloc(&d_input, N * sizeof(float));
    cudaMalloc(&d_output, N * sizeof(float));
    
    // Copy input to device
    cudaMemcpy(d_input, h_input, N * sizeof(float), cudaMemcpyHostToDevice);
    
    // Launch kernel
    safeSquare<<<blocksPerGrid, THREADS_PER_BLOCK>>>(d_input, d_output, N);
    cudaDeviceSynchronize();
    
    // Copy results back
    cudaMemcpy(h_output, d_output, N * sizeof(float), cudaMemcpyDeviceToHost);
    
    // Verify results
    printf("Results:\n");
    bool correct = true;
    for (int i = 0; i < N; i++) {
        float expected = (i + 1) * (i + 1);
        printf("  input[%d] = %.0f, output[%d] = %.0f (expected %.0f) %s\n",
               i, h_input[i], i, h_output[i], expected,
               (h_output[i] == expected) ? "✓" : "✗");
        if (h_output[i] != expected) correct = false;
    }
    printf("\n%s All results correct!\n", correct ? "✅" : "❌");
    
    // Cleanup
    cudaFree(d_input);
    cudaFree(d_output);
    
    return 0;
}

In [ ]:
!nvcc -o boundary_check boundary_check.cu && ./boundary_check

## 4. Grid-Stride Loops: The Professional Pattern

What if your array has **billions** of elements but you can only launch millions of threads?

**Grid-stride loop:** Each thread processes multiple elements, striding by the total grid size.

```
Array: [0][1][2][3][4][5][6][7][8][9][10][11]...

Grid size: 4 threads total

Thread 0: processes indices 0, 4, 8, 12, ...
Thread 1: processes indices 1, 5, 9, 13, ...
Thread 2: processes indices 2, 6, 10, 14, ...
Thread 3: processes indices 3, 7, 11, 15, ...
```

This pattern is **essential** for production code!

In [ ]:
%%writefile grid_stride_loop.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// Grid-stride loop pattern - handles ANY array size
__global__ void gridStrideSquare(float* input, float* output, int n) {
    // Starting index for this thread
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Total number of threads in the grid
    int stride = blockDim.x * gridDim.x;
    
    // Grid-stride loop: each thread handles multiple elements
    for (int i = idx; i < n; i += stride) {
        output[i] = input[i] * input[i];
    }
}

int main() {
    // Large array with limited grid
    const int N = 100000000;  // 100 million elements!
    const int THREADS_PER_BLOCK = 256;
    const int BLOCKS = 256;   // Only 256 blocks, but handles 100M elements!
    
    printf("=== Grid-Stride Loop Demo ===\n\n");
    printf("Array size:     %d elements (%.1f MB)\n", N, N * sizeof(float) / 1e6);
    printf("Grid size:      %d blocks × %d threads = %d threads\n",
           BLOCKS, THREADS_PER_BLOCK, BLOCKS * THREADS_PER_BLOCK);
    printf("Elements/thread: %.1f (average)\n\n", (float)N / (BLOCKS * THREADS_PER_BLOCK));
    
    // Allocate host memory
    float *h_input = (float*)malloc(N * sizeof(float));
    float *h_output = (float*)malloc(N * sizeof(float));
    
    // Initialize input
    srand(42);
    for (int i = 0; i < N; i++) {
        h_input[i] = (float)rand() / RAND_MAX;
    }
    
    // Allocate device memory
    float *d_input, *d_output;
    cudaMalloc(&d_input, N * sizeof(float));
    cudaMalloc(&d_output, N * sizeof(float));
    
    // Copy to device
    cudaMemcpy(d_input, h_input, N * sizeof(float), cudaMemcpyHostToDevice);
    
    // Time the kernel
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaEventRecord(start);
    gridStrideSquare<<<BLOCKS, THREADS_PER_BLOCK>>>(d_input, d_output, N);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    
    // Copy back and verify
    cudaMemcpy(h_output, d_output, N * sizeof(float), cudaMemcpyDeviceToHost);
    
    // Verify a few samples
    bool correct = true;
    for (int i = 0; i < 100 && correct; i++) {
        int idx = (i * N) / 100;  // Sample across the array
        float expected = h_input[idx] * h_input[idx];
        if (fabsf(h_output[idx] - expected) > 1e-5) {
            printf("Mismatch at %d: got %f, expected %f\n", idx, h_output[idx], expected);
            correct = false;
        }
    }
    
    // Calculate bandwidth
    float dataGB = 2.0f * N * sizeof(float) / 1e9;  // Read + write
    float bandwidth = dataGB / (milliseconds / 1000.0f);
    
    printf("⏱️  Kernel time: %.3f ms\n", milliseconds);
    printf("📊 Bandwidth:    %.1f GB/s\n", bandwidth);
    printf("%s All results correct!\n", correct ? "✅" : "❌");
    
    // Cleanup
    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    free(h_output);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    return 0;
}

In [ ]:
!nvcc -O3 -o grid_stride_loop grid_stride_loop.cu && ./grid_stride_loop

## 5. 2D Indexing: Matrices and Images

For 2D data (matrices, images), we use 2D thread blocks and grids:

```
Image (Height × Width):
┌───┬───┬───┬───┬───┐
│0,0│0,1│0,2│0,3│0,4│  Row 0
├───┼───┼───┼───┼───┤
│1,0│1,1│1,2│1,3│1,4│  Row 1
├───┼───┼───┼───┼───┤
│2,0│2,1│2,2│2,3│2,4│  Row 2
└───┴───┴───┴───┴───┘

Thread coordinates:
  row = blockIdx.y * blockDim.y + threadIdx.y
  col = blockIdx.x * blockDim.x + threadIdx.x
  
Linear index (row-major): idx = row * width + col
```

In [ ]:
%%writefile indexing_2d.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// 2D kernel to double each element of a matrix
__global__ void matrixDouble(float* matrix, int rows, int cols) {
    // Calculate 2D thread coordinates
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    
    // Boundary check for both dimensions
    if (row < rows && col < cols) {
        // Convert 2D to 1D index (row-major order)
        int idx = row * cols + col;
        matrix[idx] *= 2.0f;
    }
}

// Print matrix helper
void printMatrix(float* matrix, int rows, int cols, const char* name) {
    printf("%s:\n", name);
    for (int r = 0; r < rows; r++) {
        printf("  ");
        for (int c = 0; c < cols; c++) {
            printf("%5.0f ", matrix[r * cols + c]);
        }
        printf("\n");
    }
    printf("\n");
}

int main() {
    const int ROWS = 6;
    const int COLS = 8;
    const int SIZE = ROWS * COLS * sizeof(float);
    
    printf("=== 2D Indexing Demo ===\n\n");
    
    // Allocate and initialize host matrix
    float* h_matrix = (float*)malloc(SIZE);
    for (int i = 0; i < ROWS * COLS; i++) {
        h_matrix[i] = i;
    }
    
    printMatrix(h_matrix, ROWS, COLS, "Original matrix");
    
    // Allocate device memory
    float* d_matrix;
    cudaMalloc(&d_matrix, SIZE);
    cudaMemcpy(d_matrix, h_matrix, SIZE, cudaMemcpyHostToDevice);
    
    // Configure 2D grid
    dim3 threadsPerBlock(4, 4);  // 4×4 = 16 threads per block
    dim3 blocksPerGrid(
        (COLS + threadsPerBlock.x - 1) / threadsPerBlock.x,
        (ROWS + threadsPerBlock.y - 1) / threadsPerBlock.y
    );
    
    printf("Block size: %d × %d = %d threads\n",
           threadsPerBlock.x, threadsPerBlock.y,
           threadsPerBlock.x * threadsPerBlock.y);
    printf("Grid size:  %d × %d = %d blocks\n\n",
           blocksPerGrid.x, blocksPerGrid.y,
           blocksPerGrid.x * blocksPerGrid.y);
    
    // Launch kernel
    matrixDouble<<<blocksPerGrid, threadsPerBlock>>>(d_matrix, ROWS, COLS);
    cudaDeviceSynchronize();
    
    // Copy back and display
    cudaMemcpy(h_matrix, d_matrix, SIZE, cudaMemcpyDeviceToHost);
    printMatrix(h_matrix, ROWS, COLS, "Doubled matrix");
    
    // Verify
    bool correct = true;
    for (int i = 0; i < ROWS * COLS; i++) {
        if (h_matrix[i] != i * 2.0f) {
            correct = false;
            break;
        }
    }
    printf("%s All results correct!\n", correct ? "✅" : "❌");
    
    // Cleanup
    cudaFree(d_matrix);
    free(h_matrix);
    
    return 0;
}

In [ ]:
!nvcc -o indexing_2d indexing_2d.cu && ./indexing_2d

## 6. Practical Example: Image Grayscale Conversion

Let's apply 2D indexing to convert an RGB image to grayscale.

**Grayscale formula:** `Y = 0.299*R + 0.587*G + 0.114*B`

(These weights match human perception - we're more sensitive to green than blue.)

In [ ]:
%%writefile rgb_to_gray.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <time.h>

// GPU kernel: RGB to Grayscale conversion
__global__ void rgbToGray(unsigned char* rgb, unsigned char* gray,
                          int height, int width) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    
    if (row < height && col < width) {
        // RGB image has 3 channels per pixel
        int rgbIdx = (row * width + col) * 3;
        int grayIdx = row * width + col;
        
        // Get RGB values
        unsigned char r = rgb[rgbIdx + 0];
        unsigned char g = rgb[rgbIdx + 1];
        unsigned char b = rgb[rgbIdx + 2];
        
        // Luminance formula (human perception weighted)
        float grayVal = 0.299f * r + 0.587f * g + 0.114f * b;
        gray[grayIdx] = (unsigned char)grayVal;
    }
}

// CPU reference implementation for comparison
void rgbToGrayCPU(unsigned char* rgb, unsigned char* gray, int height, int width) {
    for (int row = 0; row < height; row++) {
        for (int col = 0; col < width; col++) {
            int rgbIdx = (row * width + col) * 3;
            int grayIdx = row * width + col;
            float grayVal = 0.299f * rgb[rgbIdx] + 0.587f * rgb[rgbIdx + 1] + 0.114f * rgb[rgbIdx + 2];
            gray[grayIdx] = (unsigned char)grayVal;
        }
    }
}

int main() {
    // Full HD image
    const int WIDTH = 1920;
    const int HEIGHT = 1080;
    const int RGB_SIZE = WIDTH * HEIGHT * 3;
    const int GRAY_SIZE = WIDTH * HEIGHT;
    
    printf("=== RGB to Grayscale Conversion ===\n\n");
    printf("Image size: %d × %d (%d pixels, %.1f MB RGB)\n",
           WIDTH, HEIGHT, WIDTH * HEIGHT, RGB_SIZE / 1e6);
    
    // Allocate host memory
    unsigned char* h_rgb = (unsigned char*)malloc(RGB_SIZE);
    unsigned char* h_gray_gpu = (unsigned char*)malloc(GRAY_SIZE);
    unsigned char* h_gray_cpu = (unsigned char*)malloc(GRAY_SIZE);
    
    // Generate random image
    srand(42);
    for (int i = 0; i < RGB_SIZE; i++) {
        h_rgb[i] = rand() % 256;
    }
    
    // Allocate device memory
    unsigned char *d_rgb, *d_gray;
    cudaMalloc(&d_rgb, RGB_SIZE);
    cudaMalloc(&d_gray, GRAY_SIZE);
    
    // Copy to device
    cudaMemcpy(d_rgb, h_rgb, RGB_SIZE, cudaMemcpyHostToDevice);
    
    // Configure 2D grid (16×16 threads per block is common for images)
    dim3 threadsPerBlock(16, 16);
    dim3 blocksPerGrid(
        (WIDTH + threadsPerBlock.x - 1) / threadsPerBlock.x,
        (HEIGHT + threadsPerBlock.y - 1) / threadsPerBlock.y
    );
    
    printf("Thread block: %d × %d = %d threads\n",
           threadsPerBlock.x, threadsPerBlock.y,
           threadsPerBlock.x * threadsPerBlock.y);
    printf("Grid: %d × %d = %d blocks\n\n",
           blocksPerGrid.x, blocksPerGrid.y,
           blocksPerGrid.x * blocksPerGrid.y);
    
    // Time GPU
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaEventRecord(start);
    rgbToGray<<<blocksPerGrid, threadsPerBlock>>>(d_rgb, d_gray, HEIGHT, WIDTH);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float gpuTime;
    cudaEventElapsedTime(&gpuTime, start, stop);
    
    // Copy back
    cudaMemcpy(h_gray_gpu, d_gray, GRAY_SIZE, cudaMemcpyDeviceToHost);
    
    // Time CPU
    clock_t cpuStart = clock();
    rgbToGrayCPU(h_rgb, h_gray_cpu, HEIGHT, WIDTH);
    clock_t cpuEnd = clock();
    float cpuTime = (float)(cpuEnd - cpuStart) / CLOCKS_PER_SEC * 1000;
    
    // Verify
    bool correct = true;
    for (int i = 0; i < GRAY_SIZE && correct; i++) {
        if (abs(h_gray_gpu[i] - h_gray_cpu[i]) > 1) {
            printf("Mismatch at %d: GPU=%d, CPU=%d\n",
                   i, h_gray_gpu[i], h_gray_cpu[i]);
            correct = false;
        }
    }
    
    printf("⏱️  GPU time: %.3f ms\n", gpuTime);
    printf("⏱️  CPU time: %.3f ms\n", cpuTime);
    printf("🚀 Speedup:  %.1fx\n", cpuTime / gpuTime);
    printf("%s Results match!\n", correct ? "✅" : "❌");
    
    // Cleanup
    cudaFree(d_rgb);
    cudaFree(d_gray);
    free(h_rgb);
    free(h_gray_gpu);
    free(h_gray_cpu);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    return 0;
}

In [ ]:
!nvcc -O3 -o rgb_to_gray rgb_to_gray.cu && ./rgb_to_gray

## 🎯 Exercises

### Exercise 1: Manual Index Calculation
Implement a kernel that adds two arrays, calculating the global index manually (don't rely on helpers).

In [ ]:
%%writefile exercise1_manual_index.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// TODO: Implement vector addition with manual index calculation
__global__ void vectorAdd(float* a, float* b, float* c, int n) {
    // TODO: Calculate global index manually using:
    // blockIdx.x, blockDim.x, threadIdx.x
    int idx = 0;  // FIX THIS!
    
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
    }
}

int main() {
    const int N = 1000;
    size_t size = N * sizeof(float);
    
    // TODO: Complete the implementation
    // 1. Allocate host and device memory
    // 2. Initialize input arrays
    // 3. Copy to device
    // 4. Launch kernel with proper configuration
    // 5. Copy back and verify
    
    printf("Exercise 1: Implement manual index calculation!\n");
    printf("Hint: idx = blockIdx.x * blockDim.x + threadIdx.x\n");
    
    return 0;
}

### Exercise 2: 2D Grid-Stride Loop
Implement a 2D grid-stride loop for processing very large images.

In [ ]:
%%writefile exercise2_2d_grid_stride.cu
#include <stdio.h>
#include <cuda_runtime.h>

// TODO: Implement 2D grid-stride loop
__global__ void processLargeImage(float* image, int height, int width) {
    // Starting position
    int startCol = blockIdx.x * blockDim.x + threadIdx.x;
    int startRow = blockIdx.y * blockDim.y + threadIdx.y;
    
    // Stride (grid size in each dimension)
    int strideX = blockDim.x * gridDim.x;
    int strideY = blockDim.y * gridDim.y;
    
    // TODO: Implement 2D grid-stride loop
    // Hint: Use nested for loops
    // for (int row = startRow; row < height; row += strideY) {
    //     for (int col = startCol; col < width; col += strideX) {
    //         // Process image[row * width + col]
    //         // Example: multiply by 2
    //     }
    // }
}

int main() {
    printf("Exercise 2: Implement 2D grid-stride loop!\n");
    printf("This pattern handles images larger than your grid size.\n");
    return 0;
}

### Exercise 3: Image Negative
Create a kernel that inverts an image (negative): `output[i] = 255 - input[i]`

In [ ]:
%%writefile exercise3_image_negative.cu
#include <stdio.h>
#include <cuda_runtime.h>

// TODO: Implement image negative kernel
__global__ void imageNegative(unsigned char* input, unsigned char* output,
                              int height, int width) {
    // TODO: Implement using 2D indexing
    // 1. Calculate row and col from thread indices
    // 2. Check boundaries
    // 3. Compute: output[idx] = 255 - input[idx]
}

int main() {
    printf("Exercise 3: Implement image negative!\n");
    printf("For each pixel: output = 255 - input\n");
    return 0;
}

## 📝 Key Takeaways

### Today You Learned:

1. **1D Indexing Formula**:
   ```cpp
   int idx = blockIdx.x * blockDim.x + threadIdx.x;
   ```

2. **2D Indexing Formula**:
   ```cpp
   int col = blockIdx.x * blockDim.x + threadIdx.x;
   int row = blockIdx.y * blockDim.y + threadIdx.y;
   int linearIdx = row * width + col;  // Row-major
   ```

3. **Always check boundaries**: `if (idx < n) { ... }`

4. **Grid-stride loops** handle any array size with fixed grid:
   ```cpp
   int idx = blockIdx.x * blockDim.x + threadIdx.x;
   int stride = blockDim.x * gridDim.x;
   for (int i = idx; i < n; i += stride) {
       // process element i
   }
   ```

5. **Common block sizes**:
   - 1D: 256 or 512 threads
   - 2D: (16, 16) or (32, 32) threads

---

### 📚 Next Up: Day 3 - Memory Fundamentals
- cudaMalloc vs cudaMallocManaged
- Pinned vs pageable memory
- Memory transfer optimization

---

### 🔗 Resources
- [Thread Hierarchy - Programming Guide](../../cuda-programming-guide/01-introduction/programming-model.md)
- [Quick Reference](../../notes/cuda-quick-reference.md)

In [ ]:
# Cleanup generated files
!rm -f thread_indexing_1d boundary_check grid_stride_loop indexing_2d rgb_to_gray
!rm -f *.cu
print("✅ Cleanup complete!")

In [ ]:
# ⚙️ Colab/Local Setup - Run this first!
# Python/Numba is OPTIONAL - for quick interactive testing only
import subprocess, sys
try:
    import google.colab
    print("🔧 Running on Google Colab - Installing dependencies...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
    print("✅ Setup complete!")
except ImportError:
    print("💻 Running locally - make sure you have: pip install numba numpy")

import numpy as np
from numba import cuda
import math

print("\n⚠️  Remember: CUDA C++ code is the PRIMARY learning material!")
print("   Python/Numba is provided for quick interactive testing only.")

# Day 2: Thread Indexing Mastery

Yesterday you launched your first CUDA kernel. Today we'll master the art of **thread indexing** - how each thread knows which data element to process.

## Learning Objectives
- Understand 1D, 2D, and 3D thread indexing
- Master the relationship between blocks, threads, and global indices
- Implement grid-stride loops for handling any array size
- Apply indexing to real 2D problems (matrices, images)

---

## 1. Thread Hierarchy Recap

### CUDA C++ Built-in Variables

| CUDA C++ | Python/Numba | Description |
|----------|--------------|-------------|
| `threadIdx.x/y/z` | `cuda.threadIdx.x/y/z` | Thread index within block |
| `blockIdx.x/y/z` | `cuda.blockIdx.x/y/z` | Block index within grid |
| `blockDim.x/y/z` | `cuda.blockDim.x/y/z` | Threads per block |
| `gridDim.x/y/z` | `cuda.gridDim.x/y/z` | Blocks in grid |

```
                           GRID
                ┌─────────────────────────┐
                │ Block   Block   Block   │
                │ (0,0)   (1,0)   (2,0)   │
                │                         │
                │ Block   Block   Block   │
                │ (0,1)   (1,1)   (2,1)   │
                └─────────────────────────┘
                        
                      Each Block:
                ┌─────────────────────────┐
                │ Thread Thread Thread    │
                │ (0,0)  (1,0)  (2,0)     │
                │                         │
                │ Thread Thread Thread    │
                │ (0,1)  (1,1)  (2,1)     │
                └─────────────────────────┘
```

**Built-in Variables (in Numba):**
- `cuda.threadIdx.x/y/z` - Thread index within block
- `cuda.blockIdx.x/y/z` - Block index within grid
- `cuda.blockDim.x/y/z` - Threads per block
- `cuda.gridDim.x/y/z` - Blocks in grid

In [ ]:
# Setup
import numpy as np
from numba import cuda
import math

print("CUDA available:", cuda.is_available())
print("Current device:", cuda.get_current_device().name.decode())

## 2. 1D Indexing: The Foundation

For 1D arrays, each thread needs a unique **global index**:

```
Global Index = blockIdx.x * blockDim.x + threadIdx.x

Example: 3 blocks × 4 threads/block = 12 threads

Block 0:  Thread 0  Thread 1  Thread 2  Thread 3
          idx=0     idx=1     idx=2     idx=3
          
Block 1:  Thread 0  Thread 1  Thread 2  Thread 3  
          idx=4     idx=5     idx=6     idx=7
          
Block 2:  Thread 0  Thread 1  Thread 2  Thread 3
          idx=8     idx=9     idx=10    idx=11
```

Let's visualize this:

In [ ]:
@cuda.jit
def print_1d_indices(output):
    """Each thread writes its indices to the output array"""
    # Get indices manually
    block_id = cuda.blockIdx.x
    thread_id = cuda.threadIdx.x
    block_size = cuda.blockDim.x
    
    # Calculate global index
    global_idx = block_id * block_size + thread_id
    
    # Or use the convenient helper:
    # global_idx = cuda.grid(1)
    
    if global_idx < output.shape[0]:
        # Store: [global_idx, block_id, thread_id]
        output[global_idx, 0] = global_idx
        output[global_idx, 1] = block_id
        output[global_idx, 2] = thread_id

# Launch with 3 blocks × 4 threads
threads_per_block = 4
blocks = 3
total_threads = blocks * threads_per_block

output = np.zeros((total_threads, 3), dtype=np.int32)
output_d = cuda.to_device(output)

print_1d_indices[blocks, threads_per_block](output_d)
result = output_d.copy_to_host()

print("1D Thread Indexing Visualization")
print("=" * 50)
print(f"Configuration: {blocks} blocks × {threads_per_block} threads/block")
print("-" * 50)
print(f"{'Global Idx':^12} | {'Block ID':^10} | {'Thread ID':^10}")
print("-" * 50)
for row in result:
    print(f"{row[0]:^12} | {row[1]:^10} | {row[2]:^10}")

## 3. Boundary Checking: When Threads > Elements

What happens when array size isn't a perfect multiple of block size?

```
Array size: 10 elements
Block size: 4 threads
Blocks needed: ceil(10/4) = 3
Total threads: 3 × 4 = 12

Thread indices: 0  1  2  3  4  5  6  7  8  9  10  11
Array elements: ✓  ✓  ✓  ✓  ✓  ✓  ✓  ✓  ✓  ✓   ✗   ✗
                                               ↑    ↑
                                          Out of bounds!
```

**Always add boundary checks!**

In [ ]:
@cuda.jit
def safe_square(input_arr, output_arr, n):
    """Square each element with proper boundary check"""
    idx = cuda.grid(1)
    
    # CRITICAL: Boundary check
    if idx < n:
        output_arr[idx] = input_arr[idx] ** 2

# Array that's NOT a multiple of block size
N = 1000
a = np.arange(N, dtype=np.float32)
b = np.zeros(N, dtype=np.float32)

threads_per_block = 256
blocks = math.ceil(N / threads_per_block)

print(f"Array size: {N}")
print(f"Threads per block: {threads_per_block}")
print(f"Blocks needed: {blocks}")
print(f"Total threads: {blocks * threads_per_block}")
print(f"Extra threads (idle): {blocks * threads_per_block - N}")

a_d = cuda.to_device(a)
b_d = cuda.to_device(b)

safe_square[blocks, threads_per_block](a_d, b_d, N)
result = b_d.copy_to_host()

# Verify
expected = a ** 2
print(f"\n✅ Correct: {np.allclose(result, expected)}")

## 4. Grid-Stride Loops: The Professional Pattern

What if your array has **billions** of elements but you can only launch millions of threads?

**Grid-stride loop**: Each thread processes multiple elements, striding by the total grid size.

```
Array: [0][1][2][3][4][5][6][7][8][9][10][11]...

Grid size: 4 threads

Thread 0: processes indices 0, 4, 8, 12, ...
Thread 1: processes indices 1, 5, 9, 13, ...
Thread 2: processes indices 2, 6, 10, 14, ...
Thread 3: processes indices 3, 7, 11, 15, ...
```

This pattern is **essential** for production code!

In [ ]:
@cuda.jit
def grid_stride_square(input_arr, output_arr, n):
    """Process arbitrary-sized arrays with grid-stride loop"""
    # Starting index for this thread
    idx = cuda.grid(1)
    
    # Total number of threads in the grid
    stride = cuda.gridsize(1)  # = blockDim.x * gridDim.x
    
    # Grid-stride loop: each thread handles multiple elements
    while idx < n:
        output_arr[idx] = input_arr[idx] ** 2
        idx += stride  # Jump to next element this thread handles

# Huge array with limited grid
N = 100_000_000  # 100 million elements!
a = np.random.randn(N).astype(np.float32)
b = np.zeros(N, dtype=np.float32)

# Fixed, reasonable grid size
threads_per_block = 256
blocks = 256  # Only 256 blocks, but handles 100M elements!

print(f"Array size: {N:,}")
print(f"Grid size: {blocks} blocks × {threads_per_block} threads = {blocks * threads_per_block:,} threads")
print(f"Elements per thread (average): {N / (blocks * threads_per_block):.1f}")

a_d = cuda.to_device(a)
b_d = cuda.to_device(b)

import time
start = time.perf_counter()
grid_stride_square[blocks, threads_per_block](a_d, b_d, N)
cuda.synchronize()
elapsed = time.perf_counter() - start

result = b_d.copy_to_host()
print(f"\n⏱️  Time: {elapsed*1000:.2f} ms")
print(f"✅ Correct: {np.allclose(result, a**2)}")

## 5. 2D Indexing: Matrices and Images

For 2D data (matrices, images), we use 2D thread blocks and grids:

```
Image (Height × Width):
┌───┬───┬───┬───┬───┐
│0,0│0,1│0,2│0,3│0,4│  Row 0
├───┼───┼───┼───┼───┤
│1,0│1,1│1,2│1,3│1,4│  Row 1
├───┼───┼───┼───┼───┤
│2,0│2,1│2,2│2,3│2,4│  Row 2
└───┴───┴───┴───┴───┘

Thread coordinates:
  row = blockIdx.y * blockDim.y + threadIdx.y
  col = blockIdx.x * blockDim.x + threadIdx.x
```

In [ ]:
@cuda.jit
def matrix_double_2d(matrix, rows, cols):
    """Double each element using 2D indexing"""
    # Get 2D thread coordinates
    col, row = cuda.grid(2)  # Returns (x, y) = (col, row)
    
    # Boundary check for both dimensions
    if row < rows and col < cols:
        matrix[row, col] *= 2

# Create a small matrix to visualize
rows, cols = 6, 8
matrix = np.arange(rows * cols, dtype=np.float32).reshape(rows, cols)

print("Original matrix:")
print(matrix)
print()

# 2D block configuration
threads_per_block_2d = (4, 4)  # 4×4 = 16 threads per block
blocks_per_grid_x = math.ceil(cols / threads_per_block_2d[0])
blocks_per_grid_y = math.ceil(rows / threads_per_block_2d[1])
blocks_per_grid_2d = (blocks_per_grid_x, blocks_per_grid_y)

print(f"Block size: {threads_per_block_2d}")
print(f"Grid size: {blocks_per_grid_2d}")

matrix_d = cuda.to_device(matrix)
matrix_double_2d[blocks_per_grid_2d, threads_per_block_2d](matrix_d, rows, cols)
result = matrix_d.copy_to_host()

print("\nDoubled matrix:")
print(result)

## 6. Practical Example: Image Processing (Grayscale)

Let's apply 2D indexing to convert an RGB image to grayscale.

Grayscale formula: `Y = 0.299*R + 0.587*G + 0.114*B`

In [ ]:
@cuda.jit
def rgb_to_grayscale(rgb_image, gray_image, height, width):
    """Convert RGB image to grayscale using 2D indexing"""
    col, row = cuda.grid(2)
    
    if row < height and col < width:
        # RGB channels
        r = rgb_image[row, col, 0]
        g = rgb_image[row, col, 1]
        b = rgb_image[row, col, 2]
        
        # Weighted sum (human perception weighting)
        gray = 0.299 * r + 0.587 * g + 0.114 * b
        
        gray_image[row, col] = gray

# Create a synthetic RGB image (like a gradient)
height, width = 1080, 1920  # Full HD
rgb_image = np.random.randint(0, 256, (height, width, 3), dtype=np.uint8).astype(np.float32)

gray_image = np.zeros((height, width), dtype=np.float32)

# Configure 2D grid
threads_per_block = (16, 16)  # 256 threads per block
blocks_x = math.ceil(width / threads_per_block[0])
blocks_y = math.ceil(height / threads_per_block[1])
blocks_per_grid = (blocks_x, blocks_y)

print(f"Image size: {height} × {width} ({height * width:,} pixels)")
print(f"Thread block: {threads_per_block}")
print(f"Grid: {blocks_per_grid}")
print(f"Total threads: {blocks_x * threads_per_block[0]} × {blocks_y * threads_per_block[1]}")

# Run on GPU
rgb_d = cuda.to_device(rgb_image)
gray_d = cuda.to_device(gray_image)

import time
start = time.perf_counter()
rgb_to_grayscale[blocks_per_grid, threads_per_block](rgb_d, gray_d, height, width)
cuda.synchronize()
gpu_time = time.perf_counter() - start

gray_result = gray_d.copy_to_host()

# Compare with NumPy (CPU)
start = time.perf_counter()
gray_cpu = 0.299 * rgb_image[:,:,0] + 0.587 * rgb_image[:,:,1] + 0.114 * rgb_image[:,:,2]
cpu_time = time.perf_counter() - start

print(f"\n⏱️  GPU time: {gpu_time*1000:.3f} ms")
print(f"⏱️  CPU time: {cpu_time*1000:.3f} ms")
print(f"🚀 Speedup: {cpu_time/gpu_time:.2f}x")
print(f"✅ Correct: {np.allclose(gray_result, gray_cpu, atol=1e-5)}")

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

Complete the exercises below in CUDA C++. These cover manual indexing, 2D grid-stride loops, and image processing.

In [ ]:
%%writefile thread_indexing_exercises.cu
// thread_indexing_exercises.cu - Thread indexing exercises
#include <stdio.h>
#include <cuda_runtime.h>
#include <stdlib.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA Error at %s:%d: %s\n", __FILE__, __LINE__, \
                    cudaGetErrorString(err)); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)

// =============================================================================
// Exercise 1: Manual Index Calculation
// Implement 1D indexing WITHOUT using cuda.grid() - calculate manually
// =============================================================================

__global__ void manualIndexAdd(const float* a, const float* b, float* c, int n) {
    // TODO: Calculate global index manually using blockIdx.x, blockDim.x, threadIdx.x
    int idx = blockIdx.x * blockDim.x + threadIdx.x;  // Manual calculation!
    
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
    }
}

// =============================================================================
// Exercise 2: 2D Grid-Stride Loop
// Process a very large matrix with 2D grid-stride loop pattern
// =============================================================================

__global__ void gridStride2D(float* matrix, int height, int width) {
    // Starting position
    int startCol = blockIdx.x * blockDim.x + threadIdx.x;
    int startRow = blockIdx.y * blockDim.y + threadIdx.y;
    
    // Stride (grid size in each dimension)
    int strideCol = blockDim.x * gridDim.x;
    int strideRow = blockDim.y * gridDim.y;
    
    // 2D grid-stride loop
    for (int row = startRow; row < height; row += strideRow) {
        for (int col = startCol; col < width; col += strideCol) {
            int idx = row * width + col;
            matrix[idx] = matrix[idx] * 2.0f;  // Double each element
        }
    }
}

// =============================================================================
// Exercise 3: Image Negative
// Invert an image: output = 255 - input
// =============================================================================

__global__ void imageNegative(const unsigned char* input, unsigned char* output, 
                               int height, int width) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    
    if (row < height && col < width) {
        int idx = row * width + col;
        output[idx] = 255 - input[idx];
    }
}

// =============================================================================
// Test harness
// =============================================================================

int main() {
    printf("=== Thread Indexing Exercises ===\n\n");
    
    // Exercise 1: Manual Index Addition
    printf("Exercise 1: Manual Index Add\n");
    printf("-" "-----------------------------\n");
    {
        const int N = 1000;
        float *h_a = (float*)malloc(N * sizeof(float));
        float *h_b = (float*)malloc(N * sizeof(float));
        float *h_c = (float*)malloc(N * sizeof(float));
        
        for (int i = 0; i < N; i++) {
            h_a[i] = (float)i;
            h_b[i] = (float)(i * 2);
        }
        
        float *d_a, *d_b, *d_c;
        CUDA_CHECK(cudaMalloc(&d_a, N * sizeof(float)));
        CUDA_CHECK(cudaMalloc(&d_b, N * sizeof(float)));
        CUDA_CHECK(cudaMalloc(&d_c, N * sizeof(float)));
        
        CUDA_CHECK(cudaMemcpy(d_a, h_a, N * sizeof(float), cudaMemcpyHostToDevice));
        CUDA_CHECK(cudaMemcpy(d_b, h_b, N * sizeof(float), cudaMemcpyHostToDevice));
        
        int threadsPerBlock = 256;
        int blocks = (N + threadsPerBlock - 1) / threadsPerBlock;
        
        manualIndexAdd<<<blocks, threadsPerBlock>>>(d_a, d_b, d_c, N);
        CUDA_CHECK(cudaGetLastError());
        CUDA_CHECK(cudaDeviceSynchronize());
        
        CUDA_CHECK(cudaMemcpy(h_c, d_c, N * sizeof(float), cudaMemcpyDeviceToHost));
        
        // Verify
        bool correct = true;
        for (int i = 0; i < N && correct; i++) {
            if (h_c[i] != h_a[i] + h_b[i]) correct = false;
        }
        printf("Result: %s\n", correct ? "✓ PASSED" : "✗ FAILED");
        printf("Sample: c[0]=%.0f, c[999]=%.0f\n\n", h_c[0], h_c[999]);
        
        cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
        free(h_a); free(h_b); free(h_c);
    }
    
    // Exercise 2: 2D Grid-Stride Loop
    printf("Exercise 2: 2D Grid-Stride Loop\n");
    printf("-" "--------------------------------\n");
    {
        const int HEIGHT = 2048;
        const int WIDTH = 2048;
        size_t size = HEIGHT * WIDTH * sizeof(float);
        
        float *h_matrix = (float*)malloc(size);
        for (int i = 0; i < HEIGHT * WIDTH; i++) {
            h_matrix[i] = (float)(i % 100);
        }
        float original_val = h_matrix[0];
        
        float *d_matrix;
        CUDA_CHECK(cudaMalloc(&d_matrix, size));
        CUDA_CHECK(cudaMemcpy(d_matrix, h_matrix, size, cudaMemcpyHostToDevice));
        
        // Use small grid to force grid-stride behavior
        dim3 threadsPerBlock(16, 16);
        dim3 blocks(8, 8);  // Only 128x128 threads for 2048x2048 matrix
        
        gridStride2D<<<blocks, threadsPerBlock>>>(d_matrix, HEIGHT, WIDTH);
        CUDA_CHECK(cudaGetLastError());
        CUDA_CHECK(cudaDeviceSynchronize());
        
        CUDA_CHECK(cudaMemcpy(h_matrix, d_matrix, size, cudaMemcpyDeviceToHost));
        
        // Verify (all elements should be doubled)
        bool correct = true;
        for (int i = 0; i < 100 && correct; i++) {
            float expected = (float)(i % 100) * 2.0f;
            if (h_matrix[i] != expected) correct = false;
        }
        printf("Result: %s\n", correct ? "✓ PASSED" : "✗ FAILED");
        printf("Sample: matrix[0]=%.0f (was %.0f)\n\n", h_matrix[0], original_val);
        
        cudaFree(d_matrix);
        free(h_matrix);
    }
    
    // Exercise 3: Image Negative
    printf("Exercise 3: Image Negative\n");
    printf("-" "--------------------------\n");
    {
        const int HEIGHT = 1080;
        const int WIDTH = 1920;
        size_t size = HEIGHT * WIDTH * sizeof(unsigned char);
        
        unsigned char *h_input = (unsigned char*)malloc(size);
        unsigned char *h_output = (unsigned char*)malloc(size);
        
        // Create gradient image
        for (int i = 0; i < HEIGHT * WIDTH; i++) {
            h_input[i] = (unsigned char)(i % 256);
        }
        
        unsigned char *d_input, *d_output;
        CUDA_CHECK(cudaMalloc(&d_input, size));
        CUDA_CHECK(cudaMalloc(&d_output, size));
        
        CUDA_CHECK(cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice));
        
        dim3 threadsPerBlock(32, 32);
        dim3 blocks((WIDTH + 31) / 32, (HEIGHT + 31) / 32);
        
        imageNegative<<<blocks, threadsPerBlock>>>(d_input, d_output, HEIGHT, WIDTH);
        CUDA_CHECK(cudaGetLastError());
        CUDA_CHECK(cudaDeviceSynchronize());
        
        CUDA_CHECK(cudaMemcpy(h_output, d_output, size, cudaMemcpyDeviceToHost));
        
        // Verify
        bool correct = true;
        for (int i = 0; i < 256 && correct; i++) {
            if (h_output[i] != (unsigned char)(255 - h_input[i])) correct = false;
        }
        printf("Result: %s\n", correct ? "✓ PASSED" : "✗ FAILED");
        printf("Sample: input[0]=%d → output[0]=%d (expected %d)\n", 
               h_input[0], h_output[0], 255 - h_input[0]);
        
        cudaFree(d_input); cudaFree(d_output);
        free(h_input); free(h_output);
    }
    
    printf("\n=== All exercises complete! ===\n");
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o thread_indexing_exercises thread_indexing_exercises.cu && ./thread_indexing_exercises

### 🔶 Python/Numba Exercises (Optional)

These exercises use Python/Numba for quick experimentation. Complete the C++ versions above first!

In [ ]:
# TODO Exercise 1: Manual indexing
@cuda.jit
def manual_index_add(a, b, c):
    """Add arrays using MANUAL index calculation"""
    # TODO: Calculate global index manually
    # idx = blockIdx.x * blockDim.x + threadIdx.x
    idx = 0  # FIX THIS
    
    if idx < c.size:
        c[idx] = a[idx] + b[idx]

# Test your implementation
# N = 1000
# a = np.random.randn(N).astype(np.float32)
# b = np.random.randn(N).astype(np.float32)
# ...

### Exercise 2: 2D Grid-Stride Loop
Implement a 2D grid-stride loop for processing very large images.

In [ ]:
# TODO Exercise 2: 2D Grid-Stride Loop
@cuda.jit
def grid_stride_2d(matrix, height, width):
    """Process large matrix with 2D grid-stride loop"""
    # Starting position
    start_col, start_row = cuda.grid(2)
    
    # Stride (grid size in each dimension)
    stride_col, stride_row = cuda.gridsize(2)
    
    # TODO: Implement 2D grid-stride loop
    # Hint: Use nested while loops
    # row = start_row
    # while row < height:
    #     col = start_col
    #     while col < width:
    #         # process matrix[row, col]
    #         col += stride_col
    #     row += stride_row
    pass

# Test with a huge matrix
# ...

### Exercise 3: Image Negative
Create a kernel that inverts an image (negative): `output[i,j] = 255 - input[i,j]`

In [ ]:
# TODO Exercise 3: Image Negative
@cuda.jit
def image_negative(input_img, output_img, height, width):
    """Invert image: output = 255 - input"""
    # TODO: Implement using 2D indexing
    pass

# Test your implementation
# ...

## 📝 Key Takeaways

### Today You Learned:

1. **1D Indexing Formula**:
   ```
   global_idx = blockIdx.x * blockDim.x + threadIdx.x
   # Or simply: cuda.grid(1)
   ```

2. **2D Indexing Formula**:
   ```
   col, row = cuda.grid(2)
   # Manually: 
   # row = blockIdx.y * blockDim.y + threadIdx.y
   # col = blockIdx.x * blockDim.x + threadIdx.x
   ```

3. **Always check boundaries**: `if idx < n:`

4. **Grid-stride loops** handle any array size with fixed grid:
   ```python
   idx = cuda.grid(1)
   stride = cuda.gridsize(1)
   while idx < n:
       # process element
       idx += stride
   ```

5. **Common block sizes**:
   - 1D: 256 or 512 threads
   - 2D: (16, 16) or (32, 32) threads

---

### 📚 Next Up: Day 3 - Memory Fundamentals
- cudaMalloc vs cudaMallocManaged
- Pinned vs pageable memory
- Memory transfer optimization

---

### 🔗 Resources
- [Thread Hierarchy - Programming Guide](../../cuda-programming-guide/01-introduction/programming-model.md)
- [Quick Reference](../../notes/cuda-quick-reference.md)